### 组建工具集

In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1726073637, 'localtime': '2024-09-11 09:53'}, 'current': {'last_updated_epoch': 1726073100, 'last_updated': '2024-09-11 09:45', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 75, 'cloud': 25, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 14.7, 'windchill_f': 58.5, 'heatindex_c': 15.1, 'heatindex_f': 59.2, 'dewpoint_c': 11.3, 'dewpoint_f': 52.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 10.6, 'gust_kph': 17.1}}"}, {'url': 'https://www.weathertab.com/en/c/e/09/un

### 使用语言模型

In [5]:
import os
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["L_OPENAI_API_BASE"],
    azure_deployment=os.environ["L_OPENAI_API_DEPLOYMENT"],
    openai_api_version=os.environ["L_OPENAI_API_VERSION"],
    openai_api_key=os.environ["L_OPENAI_API_KEY"],
)

In [6]:
from langchain_core.messages import HumanMessage
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi! I'm Bob.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [7]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_MUQhKTz8J6RtU6BP5oYOQgsq', 'type': 'tool_call'}]


### 创建 Agent

In [20]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

### 运行

In [21]:
config = {"configurable": {"thread_id": "john123"}}
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi! I'm John.")]}, config)

response["messages"]

[HumanMessage(content="Hi! I'm John.", id='d477131f-bcdd-4bb0-9187-b23abb5ac121'),
 AIMessage(content='Hello John! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 87, 'total_tokens': 98}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-2053601b-01e9-4fef-afff-476dcba4728c-0', usage_metadata={'input_tokens': 87, 'output_tokens': 11, 'total_tokens': 98})]

In [22]:
config = {"configurable": {"thread_id": "john123"}}
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="Do you know my name?")]}, config
)
response["messages"]

[HumanMessage(content="Hi! I'm John.", id='d477131f-bcdd-4bb0-9187-b23abb5ac121'),
 AIMessage(content='Hello John! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 87, 'total_tokens': 98}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-2053601b-01e9-4fef-afff-476dcba4728c-0', usage_metadata={'input_tokens': 87, 'output_tokens': 11, 'total_tokens': 98}),
 HumanMessage(content='Do you know my name?', id='0aa4a164-0811-4110-96bc-b9d2fe5bbd87'),
 AIMessage(content='Yes, you mentioned that your name is John. How can I help you, John?', response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 111, 'total_tokens': 130}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-8f823ca2-f47f-4f4d-9f36-d489708fb09e-0', usage_metadata={'input_toke

In [15]:
config = {"configurable": {"thread_id": "bob123"}}
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, config
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='4d4801c2-3d70-44d5-ab62-c51eb9874835'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WGQWzWysNojwZYVcR3a5Cn8S', 'function': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 88, 'total_tokens': 110}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-0e64d068-f0f6-4a4d-b5f1-7a0db3ca5b54-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_WGQWzWysNojwZYVcR3a5Cn8S', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 22, 'total_tokens': 110}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\

In [16]:
config = {"configurable": {"thread_id": "bob123"}}
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats my name?")]}, config
)
response["messages"]

[HumanMessage(content='whats my name?', id='a85c8c26-ddb1-4df1-af5c-0e2d9445dbbd'),
 AIMessage(content="I'm sorry, but I don't know your name. I'm an AI assistant and I don't have access to personal information.", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 86, 'total_tokens': 114}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-b53c83da-9f3e-450b-b948-1f63d15b23e9-0', usage_metadata={'input_tokens': 86, 'output_tokens': 28, 'total_tokens': 114})]